In [4]:
import midi_functions
import numpy as np

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

import pydot as pyd
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [31]:
song = '../HarryPotter.mid'
features = midi_functions.get_features(path=song)

input_length = 10 #How many past notes we want to give the LSTM to predict the next one
X, Y = midi_functions.get_XY(features,n_notes=input_length)
X = X[:,np.arange(0,4 * input_length,4)]#gets only the notes and discards velocity/time info
Y = Y[:,0]

#How many notes are present?
sorted_notes = np.copy(Y)
sorted_notes.sort()
sorted_notes = np.unique(sorted_notes)

number_of_different_notes = len(sorted_notes)
print('Number of different notes: {}'.format(number_of_different_notes))

#Convert Y into the one-hot-representation because I think that is the only thing that works with softmax representation
for i,note in enumerate(Y): #make all the notes a number between 0 and number_of_different_notes
    Y[i] = np.where(sorted_notes == note)[0]
Y = np_utils.to_categorical(Y)
print(X.shape)


Number of different notes: 13
(49, 10)


In [6]:
#randomize the two arrays and split into training and validation
n = len(X)
permutation = np.random.permutation(n)
X = X[permutation]
Y = Y[permutation]

X_val = X[:200]
Y_val = Y[:200]
X_train = X[200:]
Y_train = Y[200:]
X_train.shape

(409, 100)

In [29]:

input_dimensions = (input_length,1)

n_hidden_units = number_of_different_notes #this should probably be the number of different notes since it is also the output of softmax

model = Sequential()
model.add(LSTM(
    256,
    input_shape=(input_dimensions),
    return_sequences=True
))
#model.add(Dropout(0.3))
model.add(LSTM(n_hidden_units, return_sequences=False))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
#model.load_weights('weights.hdf5')


#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
print(model.summary())
#print(model.get_weights())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 10, 256)           264192    
_________________________________________________________________
lstm_18 (LSTM)               (None, 13)                14040     
_________________________________________________________________
activation_9 (Activation)    (None, 13)                0         
Total params: 278,232
Trainable params: 278,232
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(X.reshape(49,10,1), Y, epochs=100, batch_size=64, callbacks=callbacks_list)

Epoch 1/100
49/49 [==============================] - 2s 40ms/step - loss: 2.6439
Epoch 2/100
49/49 [==============================] - 0s 576us/step - loss: 2.5161
Epoch 3/100
49/49 [==============================] - 0s 547us/step - loss: 2.4578
Epoch 4/100
49/49 [==============================] - 0s 557us/step - loss: 2.4215
Epoch 5/100
49/49 [==============================] - 0s 555us/step - loss: 2.3858
Epoch 6/100
49/49 [==============================] - 0s 555us/step - loss: 2.3698
Epoch 7/100
49/49 [==============================] - 0s 547us/step - loss: 2.3570
Epoch 8/100
49/49 [==============================] - 0s 535us/step - loss: 2.3470
Epoch 9/100
49/49 [==============================] - 0s 526us/step - loss: 2.3395
Epoch 10/100
49/49 [==============================] - 0s 546us/step - loss: 2.3335
Epoch 11/100
49/49 [==============================] - 0s 588us/step - loss: 2.3283
Epoch 12/100
49/49 [==============================] - 0s 582us/step - loss: 2.3238
Epoch 13/100
4

Epoch 100/100
49/49 [==============================] - 0s 561us/step - loss: 2.3102
